In [9]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [10]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [11]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [12]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_15_9_6,0.997949,0.996931,0.982501,0.992251,0.002051,0.003190,0.010042,0.006414,"Hidden Size=[40], regularizer=0.01, learning_r..."
1,model_15_9_7,0.997947,0.996842,0.982425,0.992167,0.002053,0.003283,0.010086,0.006484,"Hidden Size=[40], regularizer=0.01, learning_r..."
2,model_15_9_5,0.997947,0.997028,0.982576,0.992340,0.002053,0.003089,0.009999,0.006341,"Hidden Size=[40], regularizer=0.01, learning_r..."
3,model_15_9_8,0.997944,0.996761,0.982349,0.992088,0.002056,0.003368,0.010129,0.006550,"Hidden Size=[40], regularizer=0.01, learning_r..."
4,model_15_9_4,0.997940,0.997134,0.982647,0.992433,0.002060,0.002980,0.009958,0.006264,"Hidden Size=[40], regularizer=0.01, learning_r..."
...,...,...,...,...,...,...,...,...,...,...
622,model_17_6_3,0.966613,0.949297,0.980577,0.957525,0.033387,0.080820,0.011325,0.048116,"Hidden Size=[45], regularizer=0.01, learning_r..."
643,model_13_9_1,0.964974,0.995029,0.979848,0.992621,0.035026,0.008446,0.008595,0.008516,"Hidden Size=[35], regularizer=0.01, learning_r..."
652,model_17_6_2,0.964287,0.955207,0.985949,0.963229,0.035713,0.071399,0.008193,0.041655,"Hidden Size=[45], regularizer=0.01, learning_r..."
654,model_9_9_1,0.964070,0.977960,0.962329,0.973309,0.035930,0.033936,0.040480,0.037015,"Hidden Size=[25], regularizer=0.01, learning_r..."
